In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import xgboost as xgb
import warnings

from sklearn.datasets import dump_svmlight_file
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Imputer

warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')

In [ ]:
# Read training and test data
train = pd.read_csv('../input/application_train.csv')
test = pd.read_csv('../input/application_test.csv')

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

In [ ]:
# Move the target to the last
train = train[[c for c in train if c not in ['TARGET']] + ['TARGET']]

# Split data to features and target
X_train, y_train = train.iloc[:, :-1], train.iloc[:, -1]
X_test = test.iloc[:, :]

In [ ]:
# Check the target counts
y_train.value_counts()

In [ ]:
print(type(X_train))

In [ ]:
#Data Processing
#Label Encoding

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in X_train:
    if X_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(X_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(X_train[col])
            # Transform both training and testing data
            X_train[col] = le.transform(X_train[col])
            X_test[col] = le.transform(X_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

In [ ]:
# One-hot encoding of categorical variables
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

print('Training Features shape: ', X_train.shape)
print('Testing Features shape: ', X_test.shape)

In [ ]:
# Align the training and testing data, keep only columns present in both dataframes
X_train, X_test = X_train.align(X_test, join='inner', axis=1)

print('Training Features shape: ', X_train.shape)
print('Testing Features shape: ', X_test.shape)
print(type(X_train))

In [ ]:
# Median imputation of missing values
imputer = Imputer(strategy='median')

In [ ]:
# Fit on the training data
imputer.fit(X_train)

# Transform both training and testing data
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

print(type(X_train), type(y_train))

In [ ]:

# Converting y_test to a numpy array# Conver 
y_train = y_train.as_matrix()

print(type(y_train))

In [ ]:
#training
if not os.path.exists('dtrain_sample.svm'):
    dump_svmlight_file(X_train, y_train, 'dtrain_sample.svm', zero_based=True)

dtrain = xgb.DMatrix('dtrain_sample.svm')

In [ ]:
# Training parameters (Random)
num_round = 30
param = {
    'max_depth': 3,
    'eta': 0.05,
    'silent': 1}

In [ ]:
# Make a model
model = xgb.train(param, dtrain, num_round)
model.save_model('sample_model')

In [ ]:
#prediction

dtest = xgb.DMatrix(X_test)

In [ ]:
pred = model.predict(dtest)
print(pred.shape)

In [ ]:

# Making a submission dataframe
submission = test[['SK_ID_CURR']]
submission['TARGET'] = pred

In [ ]:
# Save the submission file
submission.to_csv("submission1.csv", index=False) 